In [17]:
#!pip install pandas
#!pip install openpyxl
!pip install epitran

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 KB 1.6 MB/s eta 0:00:001.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.0 MB/s eta 0:00:0010.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 KB 9.8 MB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 KB 8.1 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10768 sha256=4e3d3c8109ea86eaaafe693ca2ca30c15426fd96c015aebdd437eca75785d686
  Stored in directory: /home/user/.cache/pip/wheels/9c/ea/66/8e45247b09052a933eb1a680b7c64802298faba58aac9b346b
Successfully built unicodecsv


In [1]:
import pandas as pd
import string
import re
import os

In [2]:
df = pd.read_csv("паспорт хорошего русского - песня скомпилированная рифма AABB.csv",)

In [3]:
path_to_file = "kino-kukushka.txt"
path_to_file = "glinka-slavsa.txt"
path_to_file = "lube-kon.txt"
path_to_file = "tscvetaeva.txt"
path_to_file = "bernes-temnaya-noc.txt"

In [4]:
with open(path_to_file) as f:
    text_song = f.readlines()
#text_song

df = pd.DataFrame(text_song, columns = ["text"])
df = df[df["text"] != "\n"]
df = df.reset_index(drop = True)

l1 = []
df2 = pd.DataFrame(columns = ["text"])
for ind in df.index:
    n1 = ind + 1
    n = n1 % 4
    if n == 1: 
        l1 = [df['text'][ind]]
    else:
        l1.append(df['text'][ind])
    if n == 0: 
        text = "".join(l1)
        new_row = pd.DataFrame([text], columns=['text'])
        df2 = pd.concat([df2, new_row], ignore_index=True)
        #text = ""

df2["text"] = df2.apply(lambda row: row["text"][:-1] if row["text"][-1] else row["text"], axis = 1)
df = df2

#df2.values

In [5]:
def get_item(text):
    arr = text.split("\n")
    #l1 = list(map(lambda x: x if x[-1].isalpha() else x[:-1], arr))
    
    def remove_punctuation(str):
        #str = str if str[-1].isalpha() else str[:-1]
        #return str.translate(str.maketrans('', '', string.punctuation))
        return re.sub(r'[^\w\s]', '', str).strip()

    arr = list(map(remove_punctuation,arr))
    return  arr
    

In [6]:
#get_item(df.iloc[0].values[0])
df["text_splitted"] = df.apply(lambda raw:get_item(raw["text"]), axis=1)

In [7]:
#df.iloc[5]["text"], df.iloc[5]["text_splitted"], 

In [8]:
def get_rhyme_score(text1, text2):

    #This function counts only the direct equals of symbols from the end of phrases 
    #and isn`t aware of fonetical equals
    
    n = 1
    while (n < len(text1) or n < len(text2)):
        c1 = text1[-n]
        c2 = text2[-n] 
        #print(c1, c2)
        if c1 == c2:
            n = n+1
        else:
            break
    return n - 1

In [9]:
"A" , get_rhyme_score(df.iloc[0]["text_splitted"][0], df.iloc[0]["text_splitted"][2])
#, get_rhyme_score(df[0][0], df[0][2]), get_rhyme_score(df[0][0], df[0][3])

('A', 0)

In [10]:
def get_rhyme_score_of_quatrain(l4_quatrain):
    l1 = l4_quatrain
    letters = "ABCD"
    n = len(letters)
    scores = [None] * n 
    for i in range(0,n):
        current_letter = letters[i]
        if scores[i] is None: 
#            scores[i] = {current_letter:100}
            scores[i] = {current_letter}
        for j in range(i+1,n):
            score = get_rhyme_score(l1[i], l1[j])
            if (score > 0):
                scores[j] = {current_letter:score}
    return scores

In [13]:
l1 = df.iloc[0]["text_splitted"]
l1 , get_rhyme_score_of_quatrain(l1)

(['Темная ночь только пули свистят по степи',
  'Только ветер гудит в проводах тускло звезды мерцают',
  'В темную ночь ты любимая знаю не спишь',
  'И у детской кроватки тайком ты слезу утираешь'],
 [{'A'}, {'B'}, {'C'}, {'C': 2}])

In [14]:
df["scores"] = df.apply(lambda raw: get_rhyme_score_of_quatrain(raw["text_splitted"]), axis = 1)

In [15]:
df

,text,text_splitted,scores
0,"Темная ночь, только пули свистят по степи\nТол...","[Темная ночь только пули свистят по степи, Тол...","[{A}, {B}, {C}, {'C': 2}]"
1,Как я люблю глубину твоих ласковых глаз\nКак я...,"[Как я люблю глубину твоих ласковых глаз, Как ...","[{A}, {B}, {C}, {'B': 3}]"
2,"Верю в тебя, в дорогую подругу мою\nЭта вера о...","[Верю в тебя в дорогую подругу мою, Эта вера о...","[{A}, {B}, {'A': 2}, {D}]"
3,"Смерть не страшна, с ней не раз мы встречались...",[Смерть не страшна с ней не раз мы встречались...,"[{A}, {B}, {C}, {D}]"


In [16]:
file_name = f"{os.path.splitext(path_to_file)[0]}.xlsx"
#file_name
df.loc[:, ["text","scores"]].to_excel(file_name)

### Let`s control the different cases:

In [ ]:
df.iloc[0].values
#here is Ok, ABAB

In [ ]:
df.iloc[1].values
#here is ABAC, should be AABB
#because phonetically ах ~ аш

In [ ]:
#df.iloc[6].values
#here is ABCC, should be AABB 
#because phonetically ных ~ ну

### Conclusion: the get_rhyme_score(text1, text2) should be extended for fonetical equal phrases
## theory here https://stihi.ru/2011/10/19/7548#:~:text=%D0%9D%D0%B5%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F%20%D1%80%D0%B8%D1%84%D0%BC%D0%B0%20%D1%85%D0%B0%D1%80%D0%B0%D0%BA%D1%82%D0%B5%D1%80%D0%B8%D0%B7%D1%83%D0%B5%D1%82%D1%81%D1%8F%20%D1%80%D0%B0%D0%B7%D0%BB%D0%B8%D1%87%D0%B8%D1%8F%D0%BC%D0%B8%20%D0%B2,%D1%81%D0%BB%D0%BE%D0%B3%D0%B0%D1%85%2C%20%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B2%20%D0%BE%D0%BA%D0%BE%D0%BD%D1%87%D0%B0%D0%BD%D0%B8%D1%8F%D1%85%20%D1%81%D1%82%D1%80%D0%BE%D0%BA.


In [ ]:
def is_Consonance():

    #https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D1%81%D0%BE%D0%BD%D0%B0%D0%BD%D1%81_(%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8F)
    return False

def is_Assonance():

    #https://en.wikipedia.org/wiki/Assonance
    return False
    


In [21]:
from epitran.backoff import Backoff

backoff = Backoff(['rus-Cyrl', 'rus-Cyrl'])
                  
backoff.transliterate('За быстроту стремительных событий,За правду, за игру…— Послушайте! — Ещё меня любите За то, что я умру')

'za bɨstrotu strʲemʲitʲelʲnɨx sobɨtʲiй,za pravdu, za iɡru…— posluʂaйtʲe! — jeɕːё mʲena lʲubʲitʲe za to, t͡ɕʲto ja umru'